In [ ]:

import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import  xml.etree.ElementTree as ET
import cv2
import shutil
from tensorflow.keras.layers import Conv2D,BatchNormalization,LeakyReLU,Flatten,Dense,Dropout,Reshape
import albumentations as A
from tensorflow.keras.callbacks import (Callback, CSVLogger, EarlyStopping, LearningRateScheduler,
                                        ModelCheckpoint, ReduceLROnPlateau)


import os

train_images="/kaggle/input/pascal-voc-2012-dataset/VOC2012_train_val/VOC2012_train_val/JPEGImages/"
train_maps="/kaggle/input/pascal-voc-2012-dataset/VOC2012_train_val/VOC2012_train_val/Annotations/"

classes=['aeroplane','bicycle','bird','boat','bottle','bus','car','cat','chair','cow','diningtable','dog','horse','motorbike','person','pottedplant','sheep','sofa','train','tvmonitor']

B=2
N_CLASSES=len(classes)
H,W=224,224
SPLIT_SIZE=int(H/32)
N_EPOCHS=135
BATCH_SIZE=32
# Data Preparation
def preprocess_xml(filename):
    tree=ET.parse(filename)
    root=tree.getroot()
    size_tree=root.find('size')
    height=float(size_tree.find('height').text)
    width=float(size_tree.find('width').text)
    bounding_boxes=[]
    class_dict={classes[i]:i for i in range(len(classes))}
    for object_tree in root.findall('object'):
        for bounding_box in object_tree.iter('bndbox'):
            xmin=float(bounding_box.find('xmin').text)
            xmax=float(bounding_box.find('xmax').text)
            ymin=float(bounding_box.find('ymin').text)
            ymax=float(bounding_box.find('ymax').text)
            break
        class_name=object_tree.find('name').text
        bounding_box=[
                (xmin+xmax)/(2*width),
                (ymin+ymax)/(2*height),
                (xmax-xmin)/width,
                (ymax-ymin)/height,
                class_dict[class_name]
        ]
        bounding_boxes.append(bounding_box)
    return tf.convert_to_tensor(bounding_boxes)


preprocess_xml(train_maps+"2007_000032.xml")
def generate_output(bounding_boxes):
  output_label=np.zeros((SPLIT_SIZE,SPLIT_SIZE,N_CLASSES+5))
  for b in range(len(bounding_boxes)):
    grid_x=bounding_boxes[...,b,0]*SPLIT_SIZE
    grid_y=bounding_boxes[...,b,1]*SPLIT_SIZE
    i=int(grid_x)
    j=int(grid_y)

    output_label[i,j,0:5]=[1.,grid_x%1,grid_y%1,bounding_boxes[...,b,2],bounding_boxes[...,b,3]]
    output_label[i,j,5+int(bounding_boxes[...,b,4])]=1.

  return tf.convert_to_tensor(output_label,tf.float32)
generate_output(preprocess_xml(train_maps+"2007_000032.xml"))[0,5]

val_list=['2007_000027.jpg','2007_000032.jpg','2007_000033.jpg','2007_000039.jpg','2007_000042.jpg','2007_000061.jpg',
          '2007_000063.jpg','2007_000068.jpg','2007_000121.jpg','2007_000123.jpg','2007_000129.jpg','2007_000170.jpg',
          '2007_000175.jpg','2007_000187.jpg','2007_000241.jpg','2007_000243.jpg','2007_000250.jpg','2007_000256.jpg',
          '2007_000272.jpg','2007_000323.jpg','2007_000332.jpg','2007_000333.jpg','2007_000346.jpg','2007_000363.jpg',
          '2007_000364.jpg','2007_000392.jpg','2007_000423.jpg','2007_000452.jpg','2007_000464.jpg','2007_000480.jpg',
          '2007_000491.jpg','2007_000504.jpg','2007_000515.jpg','2007_000528.jpg','2007_000529.jpg','2007_000549.jpg',
          '2007_000559.jpg','2007_000572.jpg','2007_000584.jpg','2007_000629.jpg','2007_000636.jpg','2007_000645.jpg',
          '2007_000648.jpg','2007_000661.jpg','2007_000663.jpg','2007_000664.jpg','2007_000676.jpg','2007_000713.jpg',
          '2007_000720.jpg','2007_000727.jpg','2007_000733.jpg','2007_000738.jpg','2007_000762.jpg','2007_000768.jpg',
          '2007_000783.jpg','2007_000793.jpg','2007_000799.jpg','2007_000804.jpg','2007_000807.jpg','2007_000822.jpg',
          '2007_001299.jpg','2007_001311.jpg','2007_001321.jpg','2007_001340.jpg']

!mkdir /kaggle/working/val_images
!mkdir /kaggle/working/val_maps
val_images='/kaggle/working/val_images/'
val_maps='/kaggle/working/val_maps/'

for name in val_list:
    shutil.copy(train_maps+name[:-3]+'xml',val_maps+name[:-3]+'xml')
    shutil.copy(train_images+name,val_images+name)




im_paths=[]
xml_paths=[]

val_im_paths=[]
val_xml_paths=[]


for i in os.listdir(train_maps):
    if((i[:-3]+'jpg') in val_list):
        continue
    im_paths.append(train_images+i[:-3]+'jpg')
    xml_paths.append(train_maps+i)



for i in os.listdir(val_maps):
    val_im_paths.append(val_images+i[:-3]+'jpg')
    val_xml_paths.append(val_maps+i)

print(len(im_paths),len(xml_paths))
print(len(val_im_paths),len(val_xml_paths))
train=tf.data.Dataset.from_tensor_slices((im_paths,xml_paths))
val=tf.data.Dataset.from_tensor_slices((val_im_paths,val_xml_paths))
for i in val.take(1):
    print(i)
def get_imbboxes(im_path,xml_path):
    img=tf.io.decode_jpeg(tf.io.read_file(im_path))
    img=tf.cast(tf.image.resize(img,size=[H,W]),dtype=tf.float32)

    bboxes=tf.numpy_function(func=preprocess_xml,inp=[xml_path],Tout=tf.float32)

    return img,bboxes
train=train.map(get_imbboxes)
val=val.map(get_imbboxes)
for i,j in val.take(1):
    print(i,j)
    cv2.imwrite('out_1.jpg',np.array(i))
transforms = A.Compose([
    A.Resize(H,W),
    A.RandomCrop(
         width=np.random.randint(int(0.9*W),W),
         height=np.random.randint(int(0.9*H),H), p=0.5),
    A.RandomScale(scale_limit=0.1, interpolation=cv2.INTER_LANCZOS4,p=0.5),
    A.HorizontalFlip(p=0.5,),
    A.Resize(H,W),

], bbox_params=A.BboxParams(format='yolo', ))
def aug_albument(image,bboxes):
  augmented=transforms(image=image,bboxes=bboxes)
  return [tf.convert_to_tensor(augmented["image"],dtype=tf.float32),
          tf.convert_to_tensor(augmented["bboxes"],dtype=tf.float32)]
def process_data(image,bboxes):
    aug= tf.numpy_function(func=aug_albument, inp=[image,bboxes], Tout=(tf.float32,tf.float32))
    return aug[0],aug[1]
train=train.map(process_data)
for i,j in train.skip(2):
  print(i.shape,j)
  break
cv2.imwrite('out_2.jpg',np.array(i))
def preprocess_augment(img,y):
  img = tf.image.random_brightness(img, max_delta=50.)
  img = tf.image.random_saturation(img, lower=0.5, upper=1.5)
  img = tf.image.random_contrast(img, lower=0.5, upper=1.5)
  #img = tf.image.random_hue(img, max_delta=0.5 )
  img = tf.clip_by_value(img, 0, 255)
  labels=tf.numpy_function(func=generate_output, inp=[y], Tout=(tf.float32))
  return img,labels
def preprocess(img,y):
  img = tf.cast(tf.image.resize(img, size=[H, W]), dtype=tf.float32)

  labels=tf.numpy_function(func=generate_output, inp=[y], Tout=(tf.float32))
  return img,labels
train=train.map(preprocess_augment)
val=val.map(preprocess)
train=(
    train
    .batch(BATCH_SIZE)
    .prefetch(tf.data.AUTOTUNE)
)

val=(
    val
    .batch(BATCH_SIZE)
    .prefetch(tf.data.AUTOTUNE)
)
# Modeling
NUM_FILTERS=512
OUTPUT_DIM=int(N_CLASSES+5*B)
base_model=tf.keras.applications.ResNet50(
    weights='imagenet',
    input_shape=(H,W,3),
    include_top=False,
)
base_model.trainable=False
model=tf.keras.Sequential([
    base_model,
    Conv2D(NUM_FILTERS,(3,3),padding='same',kernel_initializer='he_normal'),
    BatchNormalization(),
    LeakyReLU(alpha=0.1),


    Conv2D(NUM_FILTERS,(3,3),padding='same',kernel_initializer='he_normal'),
    BatchNormalization(),
    LeakyReLU(alpha=0.1),

    Conv2D(NUM_FILTERS,(3,3),padding='same',kernel_initializer='he_normal'),
    BatchNormalization(),
    LeakyReLU(),

    Flatten(),

    Dense(NUM_FILTERS,kernel_initializer='he_normal'),
    BatchNormalization(),
    LeakyReLU(alpha=0.1),


    Dense(int(SPLIT_SIZE*SPLIT_SIZE*OUTPUT_DIM),activation='sigmoid'),

    Reshape((int(SPLIT_SIZE),int(SPLIT_SIZE),OUTPUT_DIM))
])

model.summary()
def compute_iou(boxes1,boxes2):
    boxes1_t=tf.stack([boxes1[...,0]-boxes1[...,2]/2,
                       boxes1[...,1]-boxes1[...,3]/2,
                       boxes1[...,0]+boxes1[...,2]/2,
                       boxes1[...,1]+boxes1[...,3]/2],axis=-1)


    boxes2_t=tf.stack([boxes2[...,0]-boxes2[...,2]/2,
                       boxes2[...,1]-boxes2[...,3]/2,
                       boxes2[...,0]+boxes2[...,2]/2,
                       boxes2[...,1]+boxes2[...,3]/2],axis=-1)

    lu=tf.maximum(boxes1_t[...,:2],boxes2_t[...,2:])
    rd=tf.minimum(boxes1_t[...,2:],boxes2_t[...,2:])

    intersection=tf.maximum(0.0,rd-lu)
    inter_square=intersection[...,0]*intersection[...,1]

    square1=boxes1[...,2]*boxes1[...,3]
    square2=boxes2[...,2]*boxes2[...,3]

    union_square=tf.maximum(square1+square2-inter_square,1e-10)

    return tf.clip_by_value((inter_square/union_square),0.0,1.0)

def difference(x,y):
    return tf.reduce_sum(tf.square(y-x))
def yolo_loss(y_true,y_pred):
    target=y_true[...,0]
    #--------------------------------------- for object
    y_pred_extract=tf.gather_nd(y_pred,tf.where(target[:]==1))
    y_target_extract=tf.gather_nd(y_true,tf.where(target[:]==1))

    rescaler=tf.where(target[:]==1)*32

    upscaler_1=tf.concat([rescaler[:,1:],tf.zeros([len(rescaler),2],dtype=tf.int64)],axis=-1)


    target_upscaler_2=tf.repeat([[32.,32.,224.,224.]],
                                repeats=[len(rescaler)],axis=0)*tf.cast(y_target_extract[...,1:5],dtype=tf.float32)
    pred_1_upscaler_2=tf.repeat([[32.,32.,224.,224.]],repeats=[len(rescaler)],axis=0)*tf.cast(y_pred_extract[...,1:5],dtype=tf.float32)
    pred_2_upscaler_2=tf.repeat([[32.,32.,224.,224.]],repeats=[len(rescaler)],axis=0)*tf.cast(y_pred_extract[...,6:10],dtype=tf.float32)


    target_original=tf.cast(upscaler_1,dtype=tf.float32)+target_upscaler_2
    pred_1_original=tf.cast(upscaler_1,dtype=tf.float32)+pred_1_upscaler_2
    pred_2_original=tf.cast(upscaler_1,dtype=tf.float32)+pred_2_upscaler_2

    mask=tf.cast(tf.math.greater(compute_iou(target_original,pred_2_original),compute_iou(target_original,pred_1_original)),dtype=tf.int32)

    y_pred_joined=tf.transpose(tf.concat([tf.expand_dims(y_pred_extract[...,0],axis=0),tf.expand_dims(y_pred_extract[...,5],axis=0)],axis=-1))

    obj_pred=tf.gather_nd(y_pred_joined,tf.stack([tf.range(len(rescaler)),mask],axis=-1))

    object_loss=difference(tf.cast(obj_pred,dtype=tf.float32),tf.cast(tf.ones([len(rescaler)]),dtype=tf.float32))

    #------------------------------------------------------ for no object

    y_pred_extract=tf.gather_nd(y_pred[...,0:B*5],tf.where(target[:]==0))

    y_target_extract=tf.zeros(len(y_pred_extract))

    no_obj_loss_1=difference(tf.cast(y_pred_extract[...,0],dtype=tf.float32),tf.cast(y_target_extract,dtype=tf.float32))

    no_obj_loss_2=difference(tf.cast(y_pred_extract[...,5],dtype=tf.float32),tf.cast(y_target_extract,dtype=tf.float32))

    no_obj_loss=no_obj_loss_1+no_obj_loss_2

    #-------------------------------------------------------- for object class loss

    y_pred_extract=tf.gather_nd(y_pred[...,10:],tf.where(target[:]==1))
    class_extract=tf.gather_nd(y_true[...,5:],tf.where(target[:]==1))

    class_loss=difference(tf.cast(y_pred_extract,dtype=tf.float32),tf.cast(class_extract,dtype=tf.float32))

    #--------------------------------------------------------- for object center loss

    y_pred_extract=tf.gather_nd(y_pred[...,0:B*5],tf.where(target[:]==1))
    center_joined=tf.stack([y_pred_extract[...,1:3],y_pred_extract[...,6:8]],axis=1)
    center_pred=tf.gather_nd(center_joined,tf.stack([tf.range(len(rescaler)),mask],axis=-1))
    center_target=tf.gather_nd(y_true[...,1:3],tf.where(target[:]==1))


    center_loss=difference(tf.cast(center_pred,dtype=tf.float32),tf.cast(center_target,dtype=tf.float32))



    #------------------------------------------------------- for width and height

    size_joined=tf.stack([y_pred_extract[...,3:5],y_pred_extract[...,8:10]],axis=-1)

    size_pred=tf.gather_nd(size_joined,tf.stack([tf.range(len(rescaler)),mask],axis=-1))
    size_target=tf.gather_nd(y_true[...,3:5],tf.where(target[:]==1))

    size_loss=difference(tf.math.sqrt(tf.math.abs(tf.cast(size_pred,dtype=tf.float32))),tf.math.sqrt(tf.math.abs(tf.cast(size_target,dtype=tf.float32))))


    box_loss=center_loss+size_loss

    lambda_coord=5
    lambda_no_obj=0.5

    loss=object_loss+(lambda_no_obj*no_obj_loss)+tf.cast(lambda_coord*box_loss,dtype=tf.float32)+tf.cast(class_loss,dtype=tf.float32)

    return loss




#y_true=generate_output(np.array([[0.210784,0.616422,0.127451,0.232843,2]]))
y_true=generate_output(np.array([[0.509804,0.411765,0.107843,0.245098,3],
                       [0.210784,0.616422,0.127451,0.232843,2]]))

y_true=np.expand_dims(y_true,axis=0)
y_pred=np.random.normal(size = (1,7,7,30))

y_pred[0][1][4] = [0.9,0.47,0.31,0.12,0.23,   1.0,0.2,0.6,0.1,0.95,    0.9,0.8,0.2,0.6,0.1,0.5,0.9,0.35,0.9,0.8,0.2,0.6,0.1,0.5,0.9,0.35,0.1,0.5,0.9,0.35]
y_pred[0][3][2] = [0.3,0.01,0.08,0.11,0.54,   0.98,0.56,0.88,0.1,0.24, 0.09,0.018,0.22,0.16,0.01,0.05,0.99,0.3,0.09,0.018,0.22,0.16,0.01,0.05,0.99,0.3,0.01,0.05,0.99,0.3]

yolo_loss(y_true,y_pred)
!mkdir /kaggle/working/model
checkpoint_filepath='/content/drive/MyDrive/Bang/yolo_efficientnet_b1_new.h5'
callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True
)

def scheduler(epoch, lr):
  if epoch < 40:
    return 1e-3
  elif epoch>=40 and epoch<80:
    return 5e-4
  else:
    return 1e-4
lr_callback = tf.keras.callbacks.LearningRateScheduler(scheduler)
model.compile(
  loss=yolo_loss,
  optimizer=tf.keras.optimizers.Adam(1e-3),
)
history = model.fit(
  train,
  validation_data=val,
  verbose=1,
  epochs=15,
  callbacks = [lr_callback,callback]`
)
def model_test(test_path):
  try:

    print(test_path)

    img=cv2.resize(cv2.imread(test_path),(H,W))

    image=tf.io.decode_jpeg(tf.io.read_file(test_path))
    image=tf.image.resize(image, [H,W])

    output=model.predict(np.expand_dims(image, axis = 0))

    THRESH=.25

    object_positions=tf.concat(
        [tf.where(output[...,0]>=THRESH),tf.where(output[...,5]>=THRESH)],axis=0)
    print(object_positions)
    selected_output=tf.gather_nd(output,object_positions)
    print(selected_output)
    final_boxes=[]
    final_scores=[]

    for i,pos in enumerate(object_positions):
      for j in range(2):
        if selected_output[i][j*5]>THRESH:
          output_box=tf.cast(output[pos[0]][pos[1]][pos[2]][(j*5)+1:(j*5)+5],dtype=tf.float32)

          x_centre=(tf.cast(pos[1],dtype=tf.float32)+output_box[0])*32
          y_centre=(tf.cast(pos[2],dtype=tf.float32)+output_box[1])*32

          x_width,y_height=tf.math.abs(H*output_box[2]),tf.math.abs(W*output_box[3])

          x_min,y_min=int(x_centre-(x_width/2)),int(y_centre-(y_height/2))
          x_max,y_max=int(x_centre+(x_width/2)),int(y_centre+(y_height/2))

          if(x_min<=0):x_min=0
          if(y_min<=0):y_min=0
          if(x_max>=W):x_max=W
          if(y_max>=H):y_max=H
          final_boxes.append(
              [x_min,y_min,x_max,y_max,
              str(classes[tf.argmax(selected_output[...,10:],axis=-1)[i]])])
          final_scores.append(selected_output[i][j*5])
    print(final_scores)
    print('finalboxes',final_boxes)
    final_boxes=np.array(final_boxes)

    object_classes=final_boxes[...,4]
    nms_boxes=final_boxes[...,0:4]

    nms_output=tf.image.non_max_suppression(
        nms_boxes,final_scores,max_output_size=100,iou_threshold=0.2,
        score_threshold=float('-inf')
    )
    print(nms_output)

    for i in nms_output:
      cv2.rectangle(
          img,
          (int(final_boxes[i][0]),int(final_boxes[i][1])),
          (int(final_boxes[i][2]),int(final_boxes[i][3])),(0,0,255),1)
      cv2.putText(
          img,
          final_boxes[i][-1],
          (int(final_boxes[i][0]),int(final_boxes[i][1])+15),
          cv2.FONT_HERSHEY_COMPLEX_SMALL,1,(2,225,155),1
          )

    cv2.imwrite('/kaggle/working/'+filename[:-4]+'_det'+'.jpg',cv2.resize(img,(384,384)))
  except:
    print("NO object found !!!")



/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
